In [ ]:
print("Name: SUKRUTHA D")
print("SRN: PES2UG23CS622")

Name: SUKRUTHA D
SRN: PES2UG23CS622


In [ ]:
%pip install python-dotenv --upgrade --quiet langchain langchain-huggingface sentence-transformers langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


# **4a.Embeddings and Vector Spaces**

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**VIEWING A VECTOR**

In [ ]:
vector = embeddings.embed_query("Orange")

print(f"Dimensionality: {len(vector)}")
print(f"First 10 numbers: {vector[:10]}")



Dimensionality: 384
First 10 numbers: [-0.03236313909292221, 0.026842666789889336, -0.06043591350317001, 0.052247535437345505, 0.041252050548791885, -0.013370919041335583, 0.19327786564826965, -0.02527327835559845, 0.04605182260274887, -0.09115046262741089]


** MATH COSINE SIMILARITY**

In [ ]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

vec_cat = embeddings.embed_query("Cat")
vec_dog = embeddings.embed_query("Dog")
vec_car = embeddings.embed_query("Car")

print(f"Cat vs Dog: {cosine_similarity(vec_cat, vec_dog):.4f}")
print(f"Cat vs Car: {cosine_similarity(vec_cat, vec_car):.4f}")

Cat vs Dog: 0.6606
Cat vs Car: 0.4633


# **Part 4b: Navie RAG Pipeline**

In [ ]:
!pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.4 MB/s eta 0:00:00


In [ ]:
%pip install python-dotenv --upgrade --quiet faiss-cpu langchain-huggingface sentence-transformers langchain-community
from dotenv import load_dotenv
load_dotenv()
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEmbeddings
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Enter your Google API Key: ··········


**THE KNOWLEDGE BASE**

In [ ]:
from langchain_core.documents import Document

docs = [
    Document(page_content="Sam likes pasta very much with roasted garlic bread"),
    Document(page_content="The secret password to the lab is 'Pasta'."),
    Document(page_content="LangChain is a framework for developing applications powered by language models."),
]

**INDEXING**

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()

**RAG Chain**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """
Answer based ONLY on the context below:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

result = chain.invoke("What is the secret password?")
print(result)

The secret password to the lab is 'Pasta'.


# **PART 4c**

In [ ]:
import faiss
import numpy as np

d = 128
nb = 10000
xb = np.random.random((nb, d)).astype('float32')

**FLAT INDEX**

In [ ]:
index = faiss.IndexFlatL2(d)
index.add(xb)
print(f"Flat Index contains {index.ntotal} vectors")

Flat Index contains 10000 vectors


**IVF (INVERTED FILE INDEX)**

In [ ]:
nlist = 100
quantizer = faiss.IndexFlatL2(d) # The calculator for distance
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist)
index_ivf.train(xb)
index_ivf.add(xb)

In [ ]:
print("Is index trained?", index_ivf.is_trained)
print("Total vectors in index:",index_ivf.ntotal)
print("Dimensionality of index:",index_ivf.d)

Is index trained? True
Total vectors in index: 10000
Dimensionality of index: 128


In [ ]:
index_ivf.nprobe=5
xq=np.random.random((1, d)).astype('float32')
D, I = index_ivf.search(xq, 5)
print("Nearest Indices:",I)
print("Distances: ",D)

Nearest Indices: [[4659 7153 4109 3539 6586]]
Distances:  [[13.452368 13.647164 13.680537 13.718252 13.853621]]


**HNSW (Hierarchical Navigable Small World)**

In [ ]:
M = 16
index_hnsw = faiss.IndexHNSWFlat(d, M)
index_hnsw.add(xb)

In [ ]:
print("Is index trained?", index_hnsw.is_trained)
print("Total vectors in index:", index_hnsw.ntotal)
print("Dimensionality of index:", index_hnsw.d)

Is index trained? True
Total vectors in index: 10000
Dimensionality of index: 128


In [ ]:
index_hnsw.hnsw.efSearch=10
xq=np.random.random((1, d)).astype('float32')
D, I = index_hnsw.search(xq, 5)
print("Nearest Indices:",I)
print("Distances: ",D)

Nearest Indices: [[5351 2646 2277 1398 9152]]
Distances:  [[14.391083 15.675387 16.221111 16.383621 16.422215]]


**PQ (Product Quantization)**

In [ ]:
m = 8
index_pq = faiss.IndexPQ(d, m, 8)
index_pq.train(xb)
index_pq.add(xb)
print("PQ Compression complete. RAM usage minimized.")

PQ Compression complete. RAM usage minimized.


In [ ]:
print("Is index trained?", index_pq.is_trained)
print("Total vectors in index:", index_pq.ntotal)
print("Dimensionality of index:", index_pq.d)

Is index trained? True
Total vectors in index: 10000
Dimensionality of index: 128


In [ ]:
xq=np.random.random((1, d)).astype('float32')
D, I = index_pq.search(xq, 5)
print("Nearest Indices:",I)
print("Distances: ",D)

Nearest Indices: [[6895 6315  955 7723 7765]]
Distances:  [[ 9.73518  10.299735 10.435822 10.4428   10.545818]]
